In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from models.CaptionModalityClassifier import CaptionModalityClassifier
from dataset.CaptionDataModule import CaptionDataModule
from utils.caption_utils import load_embedding_matrix
import numpy as np

In [2]:
MAX_NUMBER_WORDS = 20000       # number of words to consider from embeddings vocabulary
MAX_WORDS_PER_SENTENCE = 300   # sentence maximum length
WORD_DIMENSION = 300           # number of features per embedding
NUM_CLASSES = 4                # 4 microscopy classes

DATA_PATH = '/workspace/data/multimodality_classification.csv'
EMBEDDINGS = '/workspace/data/embeddings'
BATCH_SIZE = 32

In [3]:
dm = CaptionDataModule(BATCH_SIZE, DATA_PATH, MAX_NUMBER_WORDS, MAX_WORDS_PER_SENTENCE)

In [4]:
dm.prepare_data()
dm.setup()

In [5]:
dm.vocab_size

7221

In [6]:
embeddings_dict = load_embedding_matrix(EMBEDDINGS, WORD_DIMENSION)

Dimension: 300; found 400000 word vectors.


In [7]:
if dm.vocab_size < MAX_NUMBER_WORDS:
    MAX_NUMBER_WORDS = dm.vocab_size + 1
embedding_matrix = np.zeros((MAX_NUMBER_WORDS, WORD_DIMENSION))
    
for word, idx in dm.word_index.items():    
    if idx < MAX_NUMBER_WORDS:
        word_embedding = embeddings_dict.get(word)
        if word_embedding is not None:
            embedding_matrix[idx] = word_embedding
        else:
            embedding_matrix[idx] = np.random.randn(WORD_DIMENSION)

In [8]:
model = CaptionModalityClassifier(
                 max_input_length=MAX_WORDS_PER_SENTENCE,
                 vocab_size=MAX_NUMBER_WORDS,
                 embedding_dim=WORD_DIMENSION,
                 filters=100,
                 embeddings=embedding_matrix,
                 num_classes=NUM_CLASSES,
                 train_embeddings=True)

In [14]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping

early_stop_callback = EarlyStopping(
    monitor='val_loss',
    min_delta=0.0,
    patience=5,
    verbose=True,
    mode='min'
)

trainer = Trainer(gpus=1, early_stop_callback=early_stop_callback)
trainer.fit(model, dm)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

   | Name       | Type      | Params
------------------------------------------
0  | accuracy   | Accuracy  | 0     
1  | embeddings | Embedding | 2 M   
2  | conv1d_1   | Conv1d    | 90 K  
3  | relu1      | ReLU      | 0     
4  | maxpool1   | MaxPool1d | 0     
5  | conv1d_2   | Conv1d    | 120 K 
6  | relu2      | ReLU      | 0     
7  | maxpool2   | MaxPool1d | 0     
8  | conv1d_3   | Conv1d    | 150 K 
9  | relu3      | ReLU      | 0     
10 | maxpool3   | MaxPool1d | 0     
11 | dropout    | Dropout   | 0     
12 | fc         | Linear    | 1 K   


Saving latest checkpoint..
Epoch 00008: early stopping triggered.


1